# Langchain Experiment 2 

In [ ]:
# Import Libraries and Setup
import os
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from datetime import datetime, timedelta
import re

# Set API key directly in environment
#os.environ["OPENAI_API_KEY"] = "open-ai-api-key"

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7
)

In [2]:
def extract_duration_from_response(response):
    # Extract the number of days from the response string
    match = re.search(r'(\d+) days', response)  
    if match:
        return int(match.group(1))  
    else:
        return None  

In [18]:
# Customer Proxy Prompt
customer_proxy_prompt = PromptTemplate(
    input_variables=["requirements_feedback"],
    template="""
        You are a Customer Proxy. Your role is to communicate the requirements for the face mask detection system to the Project Manager.
        Provide feedback on the project requirements and suggestions for improvement:
        - Feedback = {requirements_feedback}.
    """
)

# Project Manager Prompt
project_manager_prompt = PromptTemplate(
    input_variables=["requirements_document"],
    template="""
        You are a Project Manager. Your role is to create a comprehensive project plan for the face mask detection system based on customer requirements.
        Analyze the requirements document and outline the project plan:
        - Requirements = {requirements_document}.
        Provide a detailed project plan overview.
    """
)

# Scrum Master Prompt
scrum_master_prompt = PromptTemplate(
    input_variables=["project_update"],
    template="""
        You are a Scrum Master. Your role is to facilitate the Scrum process for the face mask detection system project.
        Based on the project update, outline the following:
        - Identify any blockers or challenges in the project.
        - Suggest action items to resolve blockers.
        - Provide the sprint number, number of user stories and a summary of the sprint progress and improvements for the next sprint:
          - Project Update = {project_update}.
    """
)

# Requirement Engineer Prompt
requirement_engineer_prompt = PromptTemplate(
    input_variables=["requirements_count"],
    template="""
        You are a Requirement Engineer. Your role is to create detailed requirements documents for the face mask detection system.
        Calculate the {{effort}} to create the requirements document in days:
        - Step 1: Work = total number of requirements = {requirements_count}.
        - Step 2: Productivity rate = 5 requirements completed every day.
        - Step 3: Effort = Work / Productivity rate.
        Provide a summary of the requirements.
    """
)

# System Engineer Prompt
system_engineer_prompt = PromptTemplate(
    input_variables=["design_pages"],
    template="""
        You are a System Engineer. Your role is to create the system architecture and detailed design documents for the face mask detection system.
        Calculate the {{effort}} to create the design document in days:
        - Step 1: Work = total number of pages = {design_pages}.
        - Step 2: Productivity rate = 5 pages completed every day.
        - Step 3: Effort = Work / Productivity rate.
        Provide an outline of the system design.
    """
)

# Software Engineer Prompt
software_engineer_prompt = PromptTemplate(
    input_variables=["sloc_count"],
    template="""
        You are a Software Engineer. Your role is to develop the face mask detection system based on the design documents.
        Calculate the {{effort}} to write the source code in days:
        - Step 1: Work = total number of lines of code (SLOC) = {sloc_count}.
        - Step 2: Productivity rate = 50 SLOC completed every day.
        - Step 3: Effort = Work / Productivity rate.
        Provide a high-level summary of the development process.
    """
)

# Test Engineer Prompt
test_engineer_prompt = PromptTemplate(
    input_variables=["test_cases"],
    template="""
        You are a Test Engineer. Your role is to create and execute test cases for the face mask detection system.
        Calculate the {{effort}} to create and execute test cases in days:
        - Step 1: Work = total number of test cases = {test_cases}.
        - Step 2: Productivity rate = 2 test cases completed every day.
        - Step 3: Effort = Work / Productivity rate.
        Provide a summary of the test plan.
    """
)

# Documentation Engineer Prompt
documentation_engineer_prompt = PromptTemplate(
    input_variables=["documentation_pages"],
    template="""
        You are a Documentation Engineer. Your role is to create user documentation and training materials for the face mask detection system.
        Calculate the {{effort}} to create documentation in days:
        - Step 1: Work = total number of documentation pages = {documentation_pages}.
        - Step 2: Productivity rate = 3 pages completed every day.
        - Step 3: Effort = Work / Productivity rate.
        Provide an outline of the documentation plan.
    """
)

# UI/UX Designer Prompt
ui_ux_designer_prompt = PromptTemplate(
    input_variables=["design_mockups"],
    template="""
        You are a UI/UX Designer. Your role is to create wireframes, mockups, and prototypes for the face mask detection system.
        Calculate the {{effort}} to create the design mockups in days:
        - Step 1: Work = total number of design mockups = {design_mockups}.
        - Step 2: Productivity rate = 2 mockups completed every day.
        - Step 3: Effort = Work / Productivity rate.
        Provide an overview of the design strategy.
    """
)

# DevOps Engineer Prompt
devops_engineer_prompt = PromptTemplate(
    input_variables=["deployment_tasks"],
    template="""
        You are a DevOps Engineer. Your role is to set up and maintain the CI/CD pipelines and infrastructure for the face mask detection system.
        Calculate the {{effort}} to deploy and manage pipelines in days:
        - Step 1: Work = total number of deployment tasks = {deployment_tasks}.
        - Step 2: Productivity rate = 4 deployment tasks completed every day.
        - Step 3: Effort = Work / Productivity rate.
        Provide an outline of the CI/CD pipeline and infrastructure plan.
    """
)

# Security Engineer Prompt
security_engineer_prompt = PromptTemplate(
    input_variables=["security_tasks"],
    template="""
        You are a Security Engineer. Your role is to identify vulnerabilities and implement security measures for the face mask detection system.
        Calculate the {{effort}} to secure the system in days:
        - Step 1: Work = total number of vulnerabilities and security features = {security_tasks}.
        - Step 2: Productivity rate = 3 tasks completed every day.
        - Step 3: Effort = Work / Productivity rate.
        Provide a summary of the security plan.
    """
)

# Business Analyst Prompt
business_analyst_prompt = PromptTemplate(
    input_variables=["feedback_items"],
    template="""
        You are a Business Analyst. Your role is to analyze customer feedback and translate it into functional requirements for the face mask detection system.
        Calculate the {{effort}} to analyze and document requirements in days:
        - Step 1: Work = total number of feedback items = {feedback_items}.
        - Step 2: Productivity rate = 4 feedback items analyzed every day.
        - Step 3: Effort = Work / Productivity rate.
        Provide a summary of the feedback analysis.
    """
)

# Technical Writer Prompt
technical_writer_prompt = PromptTemplate(
    input_variables=["documentation_pages"],
    template="""
        You are a Technical Writer. Your role is to create comprehensive user manuals, API documentation, and training materials for the face mask detection system.
        Based on the provided input, outline the following:
        - Calculate the {{effort}} to create documentation in days:
          - Step 1: Work = total number of documentation pages = {documentation_pages}.
          - Step 2: Productivity rate = 4 pages completed every day.
          - Step 3: Effort = Work / Productivity rate.
        - Highlight key sections to include in the user manual.
        - Provide an overview of the documentation strategy and delivery timeline.
    """
)




In [71]:
summary_prompt_template = """
You are the Project Manager. Based on the efforts provided by each agent for the following phases, calculate and summarize the total effort for the project:
Phases and Efforts:
- Requirement Gathering: {requirement_gathering_effort}
- Analysis and Design: {analysis_design_effort}
- Implementation: {implementation_effort}
- Testing: {testing_effort}
- Documentation: {documentation_effort}

Provide a table summarizing the phases and their respective efforts, and calculate the total effort in days.
Also provide the sprints and user stories.
"""


phase_summary_prompt = PromptTemplate(
    input_variables=["requirement_gathering_effort", "analysis_design_effort", "implementation_effort", "testing_effort", "documentation_effort", "deployment_effort"],
    template=summary_prompt_template
)


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)


phase_summary_chain = LLMChain(
    llm=llm, 
    prompt=phase_summary_prompt
)

In [20]:
# Models for all team members
customer_proxy_model = ChatOpenAI(model="gpt-4o-mini")
project_manager_model = ChatOpenAI(model="gpt-4o-mini")
scrum_master_model = ChatOpenAI(model="gpt-4o-mini")
business_analyst_model = ChatOpenAI(model="gpt-4o-mini")
ui_ux_designer_model = ChatOpenAI(model="gpt-4o-mini")
devops_engineer_model = ChatOpenAI(model="gpt-4o-mini")
security_engineer_model = ChatOpenAI(model="gpt-4o-mini")
technical_writer_model = ChatOpenAI(model="gpt-4o-mini")
tester_model = ChatOpenAI(model="gpt-4o-mini")
requirement_engineer_model = ChatOpenAI(model="gpt-4o-mini")
system_engineer_model = ChatOpenAI(model="gpt-4o-mini")
software_engineer_model = ChatOpenAI(model="gpt-4o-mini")
documentation_engineer_model = ChatOpenAI(model="gpt-4o-mini")


In [21]:
# Chains for all team members
customer_proxy_chain = LLMChain(llm=customer_proxy_model, prompt=customer_proxy_prompt)
project_manager_chain = LLMChain(llm=project_manager_model, prompt=project_manager_prompt)
scrum_master_chain = LLMChain(llm=scrum_master_model, prompt=scrum_master_prompt)
business_analyst_chain = LLMChain(llm=business_analyst_model, prompt=business_analyst_prompt)
ui_ux_designer_chain = LLMChain(llm=ui_ux_designer_model, prompt=ui_ux_designer_prompt)
devops_engineer_chain = LLMChain(llm=devops_engineer_model, prompt=devops_engineer_prompt)
security_engineer_chain = LLMChain(llm=security_engineer_model, prompt=security_engineer_prompt)
technical_writer_chain = LLMChain(llm=technical_writer_model, prompt=technical_writer_prompt)
tester_chain = LLMChain(llm=tester_model, prompt=test_engineer_prompt)
requirement_engineer_chain = LLMChain(llm=requirement_engineer_model, prompt=requirement_engineer_prompt)
system_engineer_chain = LLMChain(llm=system_engineer_model, prompt=system_engineer_prompt)
software_engineer_chain = LLMChain(llm=software_engineer_model, prompt=software_engineer_prompt)
documentation_engineer_chain = LLMChain(llm=documentation_engineer_model, prompt=documentation_engineer_prompt)


In [60]:
# Customer feedback
customer_feedback = "The system should accurately detect face masks in real-time using existing camera infrastructure."

# Project Manager's response to Customer Proxy
project_manager_response = project_manager_chain.run({"requirements_document": customer_feedback})
print("Project Manager:", project_manager_response)

# Customer Proxy's response back to the Project Manager
customer_proxy_response = customer_proxy_chain.run({"requirements_feedback": project_manager_response})
print("Customer Proxy:", customer_proxy_response)

# Business Analyst analyzes customer feedback
business_analyst_response = business_analyst_chain.run({"feedback_items": customer_feedback})
print("Business Analyst:", business_analyst_response)

# Requirement Engineer dynamically calculates the number of system requirements

requirements_count = len(business_analyst_response.split('.')) 
requirement_engineer_response = requirement_engineer_chain.run({"requirements_count": requirements_count})
print("Requirement Engineer:", requirement_engineer_response)

# UI/UX Designer dynamically calculates the number of design mockups

ui_ux_design_mockups = requirements_count // 6  
ui_ux_designer_response = ui_ux_designer_chain.run({"design_mockups": ui_ux_design_mockups})
print("UI/UX Designer:", ui_ux_designer_response)

# System Engineer dynamically calculates the number of design pages

design_pages = ui_ux_design_mockups * 2  
system_engineer_response = system_engineer_chain.run({"design_pages": design_pages})
print("System Engineer:", system_engineer_response)

# Software Engineer dynamically calculates the SLOC (Source Lines of Code)

sloc_count = design_pages * 30  
software_engineer_response = software_engineer_chain.run({"sloc_count": sloc_count})
print("Software Engineer:", software_engineer_response)

# Scrum Master checks on the project update dynamically
scrum_master_response = scrum_master_chain.run({"project_update": project_manager_response})
print("Scrum Master:", scrum_master_response)

# DevOps Engineer dynamically calculates the number of deployment tasks

deployment_tasks = sloc_count // 250  
devops_engineer_response = devops_engineer_chain.run({"deployment_tasks": deployment_tasks})
print("DevOps Engineer:", devops_engineer_response)

# Security Engineer dynamically calculates the number of security tasks

security_tasks = deployment_tasks * 2  
security_engineer_response = security_engineer_chain.run({"security_tasks": security_tasks})
print("Security Engineer:", security_engineer_response)

# Tester/QA Engineer dynamically calculates the number of test cases

design_document = system_engineer_response  

test_cases = requirements_count * 3
tester_response = tester_chain.run({"test_cases": test_cases})
print("Tester/QA Engineer:", tester_response)


# Documentation Engineer dynamically calculates the number of documentation pages

documentation_pages = requirements_count // 6 
documentation_engineer_response = documentation_engineer_chain.run({"documentation_pages": documentation_pages})
print("Documentation Engineer:", documentation_engineer_response)



Project Manager: ### Project Plan Overview for Face Mask Detection System

#### 1. **Project Title:**
   Face Mask Detection System (FMDS)

#### 2. **Project Objectives:**
   - Develop a real-time face mask detection system that leverages existing camera infrastructure.
   - Ensure high accuracy in mask detection to promote safety in public spaces.
   - Create an intuitive user interface for monitoring and reporting.

#### 3. **Project Scope:**
   - **In-Scope:**
     - Implementation of face mask detection algorithms.
     - Integration with existing camera systems.
     - Development of a user interface for monitoring.
     - Real-time data processing and analysis.
     - Reporting and alerting functionality for mask compliance.
   - **Out-of-Scope:**
     - Hardware procurement (cameras, servers).
     - Changes to existing camera infrastructure.
     - Implementation in environments without existing camera systems.

#### 4. **Stakeholders:**
   - Project Sponsor: [Name]
   - Projec

In [65]:
import re

# Function to extract duration from response
def extract_duration_from_response(response):
    match_days = re.search(r'(\d+) days', response)
    if match_days:
        return int(match_days.group(1))  # Return the number of days
    
    match_weeks = re.search(r'(\d+) weeks', response)
    if match_weeks:
        return int(match_weeks.group(1)) * 7  # Convert weeks to days
    
    return None  # If no match is found

# Define a function to calculate phase durations
def calculate_phase_duration(phase_name, chain_output):
    """
    Given a phase name and a chain output, calculate and return the duration for that phase.
    """
    duration = extract_duration_from_response(chain_output)
    return duration

# Phase 1: Requirement Gathering 
requirement_gathering_response = business_analyst_chain.run({"feedback_items": customer_feedback})
requirement_gathering_duration = calculate_phase_duration("Requirement Gathering", requirement_gathering_response)

# Phase 2: Analysis and Design 
analysis_design_response = system_engineer_chain.run({"design_pages": design_pages})
analysis_design_duration = calculate_phase_duration("Analysis and Design", analysis_design_response)
analysis_design_duration = analysis_design_duration if analysis_design_duration is not None else 1

# Phase 3: Implementation 
implementation_response = software_engineer_chain.run({"sloc_count": sloc_count})
implementation_duration = calculate_phase_duration("Implementation", implementation_response)

# Phase 4: Testing 
testing_response = tester_chain.run({"test_cases": test_cases})
testing_duration = calculate_phase_duration("Testing", testing_response)

# Phase 5: Documentation 
documentation_response = documentation_engineer_chain.run({"documentation_pages": documentation_pages})
documentation_duration = calculate_phase_duration("Documentation", documentation_response)

# Phase 6: Deployment 
deployment_response = devops_engineer_chain.run({"deployment_tasks": deployment_tasks})
deployment_duration = calculate_phase_duration("Deployment", deployment_response)
deployment_duration = deployment_duration if deployment_duration is not None else 1


phase_durations = [
    requirement_gathering_duration,
    analysis_design_duration,
    implementation_duration,
    testing_duration,
    documentation_duration,  
    deployment_duration
]

# Filter out None values before summing
valid_durations = [duration for duration in phase_durations if duration is not None]

# Calculate total project duration by summing valid durations
if valid_durations:
    total_project_duration = sum(valid_durations)
    print(f"Total Project Duration: {total_project_duration} days")


Total Project Duration: 69 days


In [72]:
# Check if documentation_duration and analysis_design_duration are None, and set default if necessary
documentation_duration = documentation_duration if documentation_duration is not None else 1  # Default to 1 if None
analysis_design_duration = analysis_design_duration if analysis_design_duration is not None else 1  # Default to 1 if None

# List of phases with their respective durations (exclude deployment phase)
phases = [
    {"phase": "Requirement Gathering", "duration": requirement_gathering_duration},
    {"phase": "Analysis and Design", "duration": analysis_design_duration},
    {"phase": "Implementation", "duration": implementation_duration},
    {"phase": "Testing", "duration": testing_duration},
    {"phase": "Documentation", "duration": documentation_duration},  # Ensure this has a valid value
]

# Initialize sprints and user stories dictionaries
sprints = {}
user_stories = {}

# Duration of each sprint (in days)
sprint_duration = 14  # 2 weeks = 14 days

# Initialize variables to track the current sprint and the total duration of the current sprint
current_sprint_number = 1
current_sprint_duration = 0
current_sprint_phases = []

# Loop through the phases and assign them to sprints
for phase in phases:
    # If the phase duration is None, set it to 1 (or another default value)
    phase_duration = phase["duration"] if phase["duration"] is not None else 1  # Default to 1 if None
    
    # If adding this phase exceeds the current sprint duration, start a new sprint
    if current_sprint_duration + phase_duration > sprint_duration:
        # Save the current sprint's phases and start a new sprint
        sprints[f"Sprint {current_sprint_number}"] = current_sprint_phases
        current_sprint_number += 1
        current_sprint_duration = 0
        current_sprint_phases = []

    # Add the current phase to the current sprint
    current_sprint_phases.append(phase["phase"])
    current_sprint_duration += phase_duration

    # Assign user stories dynamically (example: each phase has a generic user story)
    user_stories[phase["phase"]] = [f"User Story for {phase['phase']}"]

# Don't forget to add the final sprint if there are phases remaining
if current_sprint_phases:
    sprints[f"Sprint {current_sprint_number}"] = current_sprint_phases

# Run phase summary chain (excluding deployment_effort as deployment is removed)
phase_summary = phase_summary_chain.run({
    "requirement_gathering_effort": requirement_gathering_duration,
    "analysis_design_effort": analysis_design_duration,
    "implementation_effort": implementation_duration,
    "testing_effort": testing_duration,
    "documentation_effort": documentation_duration,
    "sprints": sprints,  
    "user_stories": user_stories  
})

print("Phase Summary:\n", phase_summary)


Phase Summary:
 Here's a summary of the project phases and their respective efforts in a table format:

| Phase                  | Effort (Days) |
|-----------------------|----------------|
| Requirement Gathering  | 25             |
| Analysis and Design    | 4              |
| Implementation         | 2              |
| Testing                | 5              |
| Documentation          | 33             |
| **Total Effort**      | **69**         |

### Total Effort Calculation:
- Requirement Gathering: 25 days
- Analysis and Design: 4 days
- Implementation: 2 days
- Testing: 5 days
- Documentation: 33 days

**Total Effort = 25 + 4 + 2 + 5 + 33 = 69 days**

### Sprints and User Stories
Assuming a typical Agile approach, we can break down the project into sprints and user stories. Here's an example of how you might structure them:

#### Sprint 1: Requirement Gathering
- **User Story 1**: As a stakeholder, I want to gather requirements to ensure the project meets business needs.
- **User